### Importing packages

In [51]:
import ansys.fluent.core as pyfluent
meshing_session = pyfluent.launch_fluent(precision='double', processor_count=4, mode='meshing', show_gui=True)
workflow = meshing_session.workflow
meshing = meshing_session.meshing

### Input data
insert File format (CAD or Mesh) and file path

In [52]:
import_file_format = 'CAD' # ['CAD', 'Mesh']
import_file_name = 'A:/MA_Joannis_Skempes/00_2-Personen-Buero/00_Geometrie/2-Personen-Buero_cut.scdoc' # ['<file path>']

### Import geometry

In [53]:
# initializing workflow for meshing
workflow.InitializeWorkflow(WorkflowType='Watertight Geometry')

# data input depending on file format (CAD, Mesh)
match import_file_format:
    case 'CAD':
        workflow.TaskObject['Import Geometry'].Arguments = {
            'FileFormat': import_file_format,
            'ImportType': 'Single File', # ['Single File', 'Multiple Files']
            'LengthUnit': 'mm', # ['mm', 'm', 'cm', 'in', 'ft', 'um', 'nm'] 
            'UseBodyLabels': 'No', # ['No', 'Yes']
            'FileName': import_file_name
        }
    case 'Mesh':
        workflow.TaskObject['Import Geometry'].Arguments = {
            'FileFormat': import_file_format,
            'MeshUnit': 'm', # ['m', 'cm', 'mm', 'in', 'ft', 'um', 'nm'] 
            'LengthUnit': 'mm', # ['mm', 'm', 'cm', 'in', 'ft', 'um', 'nm'] 
            'MeshFileName': import_file_name             
        }

workflow.TaskObject['Import Geometry'].Execute()

True

### Add local sizing

In [54]:
# Local Sizing: No
workflow.TaskObject['Add Local Sizing'].Arguments = {
    'AddChild': 'no' # ['no', 'yes']
}

workflow.TaskObject['Add Local Sizing'].Execute()

True

### Generate the surface mesh

In [55]:
workflow.TaskObject['Generate the Surface Mesh'].Arguments = {
    'CFDSurfaceMeshControls': {
        'MinSize': 2, # float; 
        'MaxSize': 100, # float
        'GrowthRate': 1.2, # float
        'SizeFunctions': 'Curvature & Proximity', # ['Curvature & Proximity', 'Curvature', 'Proximity'] 
        'CellsPerGap': 5, # int
        'CurvatureNormalAngle': 18, # float 
        'ScopeProximityTo': 'faces-and-edges', # ['faces-and-edges', 'faces', 'edges']
        'DrawSizeControl': True, # bool -- Toggles Draw Boxes on Screen
    },
    #'ExecuteShareTopology': 'No', # ['No', 'Yes'] -- unkloar was der Befehl macht
    'SurfaceMeshPreferences': {
        'ShowSurfaceMeshPreferences': True, # bool -- toggles advanced options
        'SMQualityImproveSkewnessLimit': 0.8, # float -- BEFEHL FEHLT NOCH
        'SMQualityCollapseLimit': 0.8, # float
        'SMQualityMaxAngle': 80, # float
        # -- Weiteren Begriffe feholen noch...
    }
}

workflow.TaskObject['Generate the Surface Mesh'].Execute()

True

### Improve surface mesh

In [56]:
workflow.TaskObject['Generate the Surface Mesh'].InsertNextTask(CommandName=r'ImproveSurfaceMesh')

workflow.TaskObject['Improve Surface Mesh'].Arguments = {
    'FaceQualityLimit': 0.4, # float
    'SMImprovePreferences': {
        'ShowSMImprovePreferences': True, # bool -- toggles advanced options
        'SIQualityMaxAngle': 160, # float
        'SIQualityIterations': 5, # int
        'SIQualityCollapseLimit': 0.6, # float
        'SIRemoveStep': 'no', # ['no', 'yes']
        # active if SIRemoveStep: 'yes -- Explanation: Are there small imperfections that can be removed?
        'SIStepQualityLimit': 0, #float
        'SIStepWidth': 0 #float
    }
}
workflow.TaskObject['Improve Surface Mesh'].Execute()

True

### Describe geometry

In [57]:
input1 = 'fluids'
print("Geometry= " + input1)

#vorher damit, Capping egal ist wenn fluids eingestellt werden...
workflow.TaskObject['Describe Geometry'].Arguments = {
    'CappingRequired': 'No', # ['No', 'Yes'] #nur Yes wenn SOLIDS/VOIDS vorhanden!!!
    'WallToInternal': 'No', # ['No', 'Yes']
    'InvokeShareTopology': 'No', # ['No', 'Yes']
    'Multizone': 'No', # ['No', 'Yes']
}

match input1: #wenn es nicht matched: keine Änderung
    case "solids":
        workflow.TaskObject['Describe Geometry'].Arguments = {
            'SetupType': 'The geometry consists of only solid regions'
        }
    case "fluids":
        workflow.TaskObject['Describe Geometry'].Arguments = {
            'SetupType': 'The geometry consists of only fluid regions with no voids'
        }
    case "fluids-solids-voids":
        workflow.TaskObject['Describe Geometry'].Arguments = {
            'SetupType': 'The geometry consists of both fluid and solid regions and/or voids'
        }
        
workflow.TaskObject['Describe Geometry'].Execute()

Geometry= fluids


True

#### Updating boundarys

In [49]:
test = meshing_session.scheme_eval.scheme_eval('(tgapi-util-convert-zone-ids-to-name-strings (get-face-zones-of-filter "*let*"))')

print(test)

None
Shutting down Server...


In [80]:
workflow.TaskObject['Update Boundaries'].Arguments = {
    'SelectionType': 'label', #label, zone
    #'ListAllBoundariesToggle': True, #kein Einfluss?
    'OldBoundaryLabelList': ['int-anf', 'int-z-last-fenster', 'int-z-last-lampe1', 'int-z-last-lampe2', 'int-z-last-monitor1', 'int-z-last-monitor2', 'int-z-last-pers1', 'int-z-last-pers2', 'int-z-last-rechner1'], #Namenkonvention die rein kommt
    'BoundaryLabelList': ['int-anf', 'int-z-last-fenster', 'int-z-last-lampe1', 'int-z-last-lampe2', 'int-z-last-monitor1', 'int-z-last-monitor2', 'int-z-last-pers1', 'int-z-last-pers2', 'int-z-last-rechner1'],
    'BoundaryLabelTypeList': ['internal', 'internal', 'internal', 'internal', 'internal', 'internal', 'internal', 'internal', 'internal'], #Typ zuordnen
    'OldBoundaryLabelList': ['po-tuer', 'vi-ab', 'vi-zu'], #Namenkonvention die rein kommt
    'BoundaryLabelList': ['po-tuer', 'vi-ab', 'vi-zu'],
    'BoundaryLabelTypeList': ['pressure-outlet', 'velocity-inlet', 'velocity-inlet'], #Typ zuordnen
}

#create-regions ist noch ausstehend...

#workflow.TaskObject['Update Boundaries'].Execute()

### Updating regions

In [134]:
workflow.TaskObject['Update Regions'].Arguments = {
    'RegionNameList': ['fluid'],
    'RegionTypeList': ['fluid'],
}

workflow.TaskObject['Update Regions'].Execute()

True

### Add boundary layers

In [165]:
# Unklar, owzu notwendig
#workflow.TaskObject['Add Boundary Layers'].Arguments.set_state({r'LocalPrismPreferences': {r'Continuous': r'Stair Step',},})

workflow.TaskObject['Add Boundary Layers'].Arguments = {
    'AddChild': 'yes', # ['yes', 'no', 'yes, using prism controlfile']
    'BLControlName': 'smooth-transition_001',
    'OffsetMethodType': 'smooth-transition', # ['smooth-transition', 'uniform', 'last-ratio', 'aspect-ratio']
    'NumberOfLayers': 5, # int
    'TransitionRatio': 0.272, # float
    'Rate': 1.2, #float
    'FirstAspectRatio': 5, # int; bei last-ratio
    'FirstHeight': 0.04, # float
    #'AddIn': 'fluid-regions', #Befehl noch unklar...
    #'GrowOn': 'only-walls', #Befehl noch unklar...
    'LocalPrismPreferences': {
        'ShowLocalPrismPreferences': True,
        'IgnoreBoundaryLayers': 'yes', # ['yes', 'no']
        'AdditionalIgnoredLayers': 0, # int
        'ModifyAtInvalidNormals': 'no', # ['no', 'yes']
        'SphereRadiusFactorAtInvalidNormals': 0.9, # float
        'AllowedTangencyAtInvalidNormals': 0.88, # float
        'RemeshAtInvalidNormals': 'no',  # ['no', 'yes']                         
    }
    #Add-in: ????
    #Grow-on: ????
    #Advanced-options: ????
}

#Zum ausführen der boundary layers notwendig --> Hinzufügen Layer und updaten
workflow.TaskObject['Add Boundary Layers'].AddChildAndUpdate()

True

### Generate the volume mesh

In [76]:
solver_type = 'Fluent' # ['Fluent', 'CFX']
cell_type = 'polyhedra' # ['polyhedra', 'poly-hexcore', 'hexcore', 'tetrahedral']
HexMinCellLength_var = [0.01, 0.02, 0.03, 0.04, 0.05]

match solver_type:
    case 'Fluent':
        match cell_type:
            case 'poly-hexcore':
                workflow.TaskObject['Generate the Volume Mesh'].Arguments = {
                    'Solver': solver_type,
                    'VolumeFill': cell_type, # ['polyhedra', 'poly-hexcore', 'hexcore', 'tetrahedral']
                    'VolumeFillControls': {
                        'BufferLayers': 2, # int
                        'PeelLayers': 1, # int
                        'HexMinCellLength': HexMinCellLength_var[i], # float
                        'HexMaxCellLength': 0.2, # float          
                    },
                    'ParallelMeshing': True, # bool
                    # Advanced Options
                    'VolumeMeshPreferences': {
                        'ShowVolumeMeshPreferences': True, # bool
                        'QualityMethod': 'Orthogonal', # ['Orthogonal', 'Enhanced Orthogonal', 'Skewness']
                        'QualityWarningLimit': 0.05, # float
                        'UseSizeField': 'no', # ['no', 'yes']
                        'PolyFeatureAngle': 30, # float 
                        'Avoid1_8Transition': 'no', # ['no', 'yes']
                        'CheckSelfProximity': 'no', # ['no', 'yes'] 
                        'WritePrismControlFile': 'no', # ['no', 'yes']
                    },
                    # Global Boundary Layer Settings
                    'PrismPreferences': {
                        'ShowPrismPreferences': True, # bool
                        'MergeBoundaryLayers': 'yes', # ['yes', 'no']
                        'PrismGapFactor': 0.25, # float
                        'PrismMaxAspectRatio': 25, # int
                        'PrismMinAspectRatio': 1, # float
                        'PrismKeepFirstLayer': 'no', # ['no', 'yes']
                        'PrismAdjacentAngle': 80.1, # float
                        'PrismStairStepOptions': 'Yes' # ['Yes', 'No, Exclude proximity check', 'No, Exclude quality check', 'No, Exclude both checks'] 
                    }
                }
                    
            case 'polyhedra':
                workflow.TaskObject['Generate the Volume Mesh'].Arguments = {
                    'Solver': solver_type,
                    'VolumeFill': cell_type, # ['polyhedra', 'poly-hexcore', 'hexcore', 'tetrahedral']
                    'VolumeFillControls': {
                        'BufferLayers': 2, # int
                        'PeelLayers': 1, # int
                        'HexMinCellLength': HexMinCellLength_var[i], # float
                        'HexMaxCellLength': 0.2, # float
                    },
                    'ParallelMeshing': True, # bool
                    # Advanced Options
                    'VolumeMeshPreferences': {
                        'ShowVolumeMeshPreferences': True, # bool
                        'QualityMethod': 'Orthogonal', # ['Orthogonal', 'Enhanced Orthogonal', 'Skewness']
                        'QualityWarningLimit': 0.05, # float
                        'UseSizeField': 'no', # ['no', 'yes']
                        'PolyFeatureAngle': 30, # float 
                        'Avoid1_8Transition': 'no', # ['no', 'yes']
                        'CheckSelfProximity': 'no', # ['no', 'yes'] 
                        'WritePrismControlFile': 'no', # ['no', 'yes']
                    },
                    # Global Boundary Layer Settings
                    'PrismPreferences': {
                        'ShowPrismPreferences': True, # bool
                        'MergeBoundaryLayers': 'yes', # ['yes', 'no']
                        'PrismGapFactor': 0.25, # float
                        'PrismMaxAspectRatio': 25, # int
                        'PrismMinAspectRatio': 1, # float
                        'PrismKeepFirstLayer': 'no', # ['no', 'yes']
                        'PrismAdjacentAngle': 80.1, # float
                        'PrismStairStepOptions': 'Yes' # ['Yes', 'No, Exclude proximity check', 'No, Exclude quality check', 'No, Exclude both checks'] 
                    }
                }
                    
            case 'hexcore':
                workflow.TaskObject['Generate the Volume Mesh'].Arguments = {
                    'Solver': solver_type,
                    'VolumeFill': cell_type, # ['polyhedra', 'poly-hexcore', 'hexcore', 'tetrahedral']
                    'VolumeFillControls': {
                        'BufferLayers': 2, # int
                        'PeelLayers': 1, # int
                        'HexMinCellLength': HexMinCellLength_var[i], # float
                        'HexMaxCellLength': 0.2, # float
                    },
                    'ParallelMeshing': True, # bool
                    # Advanced Options
                    'VolumeMeshPreferences': {
                        'ShowVolumeMeshPreferences': True, # bool
                        'QualityMethod': 'Orthogonal', # ['Orthogonal', 'Enhanced Orthogonal', 'Skewness']
                        'QualityWarningLimit': 0.05, # float
                        'UseSizeField': 'no', # ['no', 'yes']
                        'PolyFeatureAngle': 30, # float 
                        'Avoid1_8Transition': 'no', # ['no', 'yes']
                        'CheckSelfProximity': 'no', # ['no', 'yes'] 
                        'WritePrismControlFile': 'no', # ['no', 'yes']
                    },
                    # Global Boundary Layer Settings
                    'PrismPreferences': {
                        'ShowPrismPreferences': True, # bool
                        'MergeBoundaryLayers': 'yes', # ['yes', 'no']
                        'PrismGapFactor': 0.25, # float
                        'PrismMaxAspectRatio': 25, # int
                        'PrismMinAspectRatio': 1, # float
                        'PrismKeepFirstLayer': 'no', # ['no', 'yes']
                        'PrismAdjacentAngle': 80.1, # float
                        'PrismStairStepOptions': 'Yes' # ['Yes', 'No, Exclude proximity check', 'No, Exclude quality check', 'No, Exclude both checks'] 
                    }
                }

            case 'tetrahedral':
                workflow.TaskObject['Generate the Volume Mesh'].Arguments = {
                    'Solver': solver_type,
                    'VolumeFill': cell_type, # ['polyhedra', 'poly-hexcore', 'hexcore', 'tetrahedral']
                    'VolumeFillControls': {
                        'BufferLayers': 2, # int
                        'PeelLayers': 1, # int
                        'HexMinCellLength': HexMinCellLength_var[i], # float
                        'HexMaxCellLength': 0.2, # float
                    },
                    'ParallelMeshing': True, # bool
                    # Advanced Options
                    'VolumeMeshPreferences': {
                        'ShowVolumeMeshPreferences': True, # bool
                        'QualityMethod': 'Orthogonal', # ['Orthogonal', 'Enhanced Orthogonal', 'Skewness']
                        'QualityWarningLimit': 0.05, # float
                        'UseSizeField': 'no', # ['no', 'yes']
                        'PolyFeatureAngle': 30, # float 
                        'Avoid1_8Transition': 'no', # ['no', 'yes']
                        'CheckSelfProximity': 'no', # ['no', 'yes'] 
                        'WritePrismControlFile': 'no', # ['no', 'yes']
                    },
                    # Global Boundary Layer Settings
                    'PrismPreferences': {
                        'ShowPrismPreferences': True, # bool
                        'MergeBoundaryLayers': 'yes', # ['yes', 'no']
                        'PrismGapFactor': 0.25, # float
                        'PrismMaxAspectRatio': 25, # int
                        'PrismMinAspectRatio': 1, # float
                        'PrismKeepFirstLayer': 'no', # ['no', 'yes']
                        'PrismAdjacentAngle': 80.1, # float
                        'PrismStairStepOptions': 'Yes' # ['Yes', 'No, Exclude proximity check', 'No, Exclude quality check', 'No, Exclude both checks'] 
                    }
                }
}

#workflow.TaskObject['Generate the Volume Mesh'].Execute()

SyntaxError: unmatched '}' (3629118185.py, line 147)

In [206]:
solver1 = 'Fluent' #Fluent, CFX
#Bei CFX nur hexcore und tetrahedral zur Auswahl

workflow.TaskObject['Generate the Volume Mesh'].Arguments = {
    'Solver': solver1,
    'VolumeFill': 'hexcore', #polyhedra #poly-hexcore #hexcore #tetrahedral
    'VolumeFillControls': {
        'GrowthRate': 1.2, #float
        'TetPolyMaxCellLength': 0.26, #float
        'BufferLayers': 2, #int
        'PeelLayers': 1, #int
        'HexMinCellLength': 0.02, #float; bei hexcore
        'HexMaxCellLength': 0.16, #float
    },
    'VolumeMeshPreferences': {
        'ShowVolumeMeshPreferences': True,
        'QualityMethod': 'Orthogonal',
        'QualityWarningLimit': 0.05,
        'UseSizeField': 'no', #yes, no
        'PolyFeatureAngle': 30, #yes, no
        'Avoid1_8Transition': 'no', #yes, no
        'CheckSelfProximity': 'no', #yes, no
        'WritePrismControlFile': 'no', #yes, no
        #'PolyInSolids': 'no', #yes, no
        #'SolidGrowthRate': 1.5, #aktiv, wenn PolyInSolids 'yes'
        #'MergeBodyLabels': 'no', #unklar was Befehl macht
        #'PrepareZoneNames': 'no', #unklar was Befehl macht
        },
    'ParallelMeshing': True,
    #AdcancedOptions (für alle gleich)
    'PrismPreferences': {
        'ShowPrismPreferences': True, #bool
        'MergeBoundaryLayers': 'yes', #yes, no
        'PrismGapFactor': 0.25, #float
        'PrismMaxAspectRatio': 25, #int
        'PrismMinAspectRatio': 1, #int
        'PrismKeepFirstLayer': 'no', #yes, no
        'PrismAdjacentAngle': 80, #deg
        'PrismStairStepOptions': 'Yes', #Yes #No, Exclude proximity check #No, Exclude quality check #No, Exclude both checks
        #'IgnoreInflation': 'no', #unklar was Befehl macht
    }
}

#workflow.TaskObject['Generate the Volume Mesh'].Execute()

True

### Exiting solver

In [ ]:
#solver.exit()